In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Function to extract product links from a category page
def get_product_links(category_url, num_pages):
    product_links = []
    for page in range(1, num_pages + 1):
        try:
            driver.get(f"{category_url}?page={page}")
            time.sleep(5)  # Wait for the page to load
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Update the selector to match Konga's HTML structure for product links
            for link in soup.select('div._7e903_3FsI6 a'):
                product_links.append('https://www.konga.com' + link['href'])
        except Exception as e:
            print(f"Error on page {page} of {category_url}: {e}")
            continue
    
    return product_links

# Function to extract reviews and ratings from a product page
def get_reviews(product_url):
    reviews = []
    try:
        driver.get(product_url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Update the selector to match Konga's HTML structure for reviews
        review_elements = soup.select('div.cfc8e_RM54f > div')
        for element in review_elements:
            review_text = element.select_one('p.a397c_2uBaY').get_text(strip=True)
            # Count filled stars
            filled_stars = len(element.select('svg.ba6f2_18Jb4[fill="#DBDBDB"]'))
            reviews.append({'product_url': product_url, 'review_text': review_text, 'rating': filled_stars})
    
    except Exception as e:
        print(f"Error extracting reviews from {product_url}: {e}")
    
    return reviews

# Main function to scrape reviews from a single category
def scrape_konga_fashion_reviews():
    category_url = 'https://www.konga.com/category/konga-fashion-1259'
    num_pages = 1  # For a small test, scrape only the first 2 pages
    all_reviews = []

    print(f"Scraping category: Fashion")
    product_links = get_product_links(category_url, num_pages)
    
    for product_url in product_links:
        reviews = get_reviews(product_url)
        for review in reviews:
            review['category'] = 'Fashion'  # Add category to each review
        all_reviews.extend(reviews)
        
        # Save progress periodically
        if len(all_reviews) >= 10:  # Adjusted for a small test
            df = pd.DataFrame(all_reviews)
            df.to_csv('konga_fashion_reviews.csv', mode='a', header=False, index=False)
            all_reviews = []  # Reset the list to save memory

    # Save any remaining reviews
    if all_reviews:
        df = pd.DataFrame(all_reviews)
        df.to_csv('konga_fashion_reviews.csv', mode='a', header=False, index=False)

    # Close the driver
    driver.quit()

# Run the scraping process for the Fashion category
scrape_konga_fashion_reviews()


Scraping category: Fashion


In [3]:
nlp_sample_konga.ipynb

NameError: name 'nlp_sample_konga' is not defined